In [1]:
import praw
import pandas as pd
import time
from datetime import datetime, timedelta
import csv  # for quoting

# --- Logging ---
def log(msg):
    print(msg)

def timestamp():
    return datetime.now().strftime('%H:%M:%S')

# --- Reddit API setup ---
reddit = praw.Reddit(
    client_id="0GgjEdXVoxIm0LTkBxIjNA",
    client_secret="NsY4t_AN6o5E4SSki7YkhVee07DCzw",
    user_agent="game-review-bot"
)

# --- Game and keywords ---
game_name = "Kingdom Come: Deliverance 2"
subreddit = "shouldIbuythisgame"

keywords = [
    "Kingdom come: deliverance", "kcd2"
]

# --- Date range: daily ---
start_date = datetime(2025, 2, 28)
end_date = datetime(2025, 3, 31)
step = timedelta(days=1)

# --- Storage ---
posts_data = []
comments_data = []
deduped_post_ids = set()

# --- Daily scraping loop ---
current_start = start_date
while current_start < end_date:
    current_end = current_start + step
    start_ts = int(current_start.timestamp())
    end_ts = int(current_end.timestamp())

    log(f"[{timestamp()}] 🔍 Scraping for {game_name} in r/{subreddit} from {current_start.date()} to {current_end.date()}")

    try:
        for kw in keywords:
            for post in reddit.subreddit(subreddit).search(kw, sort="new", time_filter="all", limit=100):
                time.sleep(1)

                if not (start_ts <= post.created_utc < end_ts):
                    continue
                if post.id in deduped_post_ids:
                    continue

                full_text = f"{post.title} {post.selftext}".lower()
                title_body_keywords = [k for k in keywords if k in full_text]
                is_strong_match = kw in full_text

                if is_strong_match or len(title_body_keywords) >= 2:
                    deduped_post_ids.add(post.id)
                    log(f"✅ Matched post: {post.title[:80]}...")

                    posts_data.append({
                        "game": game_name,
                        "keyword": kw,
                        "subreddit": subreddit,
                        "post_id": post.id,
                        "title": post.title,
                        "selftext": post.selftext,
                        "score": post.score,
                        "num_comments": post.num_comments,
                        "created_utc": post.created_utc,
                        "author": str(post.author),
                        "matched_keywords_title_body": ", ".join(title_body_keywords)
                    })

                    # --- Save all comments ---
                    try:
                        post.comments.replace_more(limit=None)
                        all_comments = post.comments.list()
                        log(f"    🗨️ {len(all_comments)} comments in post {post.id}")

                        for comment in all_comments[:300]:
                            try:
                                comments_data.append({
                                    "game": game_name,
                                    "keyword": kw,
                                    "subreddit": subreddit,
                                    "post_id": post.id,
                                    "comment_id": comment.id,
                                    "comment_body": comment.body,
                                    "author": str(comment.author) if comment.author else "deleted",
                                    "score": comment.score,
                                    "created_utc": comment.created_utc
                                })
                            except Exception as e:
                                log(f"⚠️ Skipped bad comment in post {post.id}: {e}")
                                continue

                    except Exception as e:
                        log(f"⚠️ Error fetching comments for post {post.id}: {e}")

    except Exception as e:
        log(f"⛔ Error scraping r/{subreddit} on {current_start.date()} → {e}")

    current_start = current_end  # move to next day

# --- Save data to CSV with clean column formatting ---
posts_df = pd.DataFrame(posts_data).drop_duplicates(subset="post_id")
comments_df = pd.DataFrame(comments_data)

log(f"\n✅ Done! Saved {len(posts_df)} posts and {len(comments_df)} comments.")

[01:42:32] 🔍 Scraping for Kingdom Come: Deliverance 2 in r/shouldIbuythisgame from 2025-02-28 to 2025-03-01
[01:44:46] 🔍 Scraping for Kingdom Come: Deliverance 2 in r/shouldIbuythisgame from 2025-03-01 to 2025-03-02
[01:46:59] 🔍 Scraping for Kingdom Come: Deliverance 2 in r/shouldIbuythisgame from 2025-03-02 to 2025-03-03
[01:49:13] 🔍 Scraping for Kingdom Come: Deliverance 2 in r/shouldIbuythisgame from 2025-03-03 to 2025-03-04
[01:51:28] 🔍 Scraping for Kingdom Come: Deliverance 2 in r/shouldIbuythisgame from 2025-03-04 to 2025-03-05
[10:19:18] 🔍 Scraping for Kingdom Come: Deliverance 2 in r/shouldIbuythisgame from 2025-03-05 to 2025-03-06
✅ Matched post: Do I get this...
    🗨️ 21 comments in post 1j476vm
[10:21:32] 🔍 Scraping for Kingdom Come: Deliverance 2 in r/shouldIbuythisgame from 2025-03-06 to 2025-03-07
[10:23:46] 🔍 Scraping for Kingdom Come: Deliverance 2 in r/shouldIbuythisgame from 2025-03-07 to 2025-03-08
[10:25:59] 🔍 Scraping for Kingdom Come: Deliverance 2 in r/shouldIbu

In [8]:
# --- Save Comments and Posts Data ---
posts_df = pd.DataFrame(posts_data)
comments_df = pd.DataFrame(comments_data)

posts_df.to_csv("Sibtg_posts_KCD2_att2.csv", index=False)
comments_df.to_csv("Sibtg_comments_KCD2_att2.csv", index=False)

log(f"\n✅ Done! Saved {len(posts_df)} posts and {len(comments_df)} comments.")


✅ Done! Saved 10 posts and 162 comments.
